In [1]:
pip install pyodbc


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: nltk in c:\users\owner\appdata\local\programs\python\python310\lib\site-packages (3.9.1)




[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import pandas as pd
import pyodbc
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [4]:
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Owner\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [5]:
text = 'I love the book very much'
score = sia.polarity_scores(text)
score

{'neg': 0.0, 'neu': 0.488, 'pos': 0.512, 'compound': 0.6369}

In [7]:
compound = score['compound']
compound

0.6369

In [6]:
# establish connection between sql and python
conn = pyodbc.connect(
'Driver={ODBC Driver 17 for SQL Server};'
'Server=localhost\\SQLEXPRESS;'
'Database=PortfolioProject_MarketingAnalytics;'
'Trusted_Connection=yes;'
)

In [7]:
query = """
 SELECT [ReviewID]
      ,[ReviewDate]
      ,[CustomerID]
      ,[ProductID]
      ,[Rating]
      ,REPLACE(ReviewText,'  ',' ') as ReviewText
  FROM [PortfolioProject_MarketingAnalytics].[dbo].[customer_reviews]
"""
df= pd.read_sql (query,conn)
df.head()

C:\Users\Owner\AppData\Local\Temp\ipykernel_12312\1763793178.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df= pd.read_sql (query,conn)


,ReviewID,ReviewDate,CustomerID,ProductID,Rating,ReviewText
0,1,2023-12-23,77,18,3,"Average experience, nothing special."
1,2,2024-12-25,80,19,5,The quality is top-notch.
2,3,2025-01-26,50,13,4,Five stars for the quick delivery.
3,4,2025-04-21,78,15,3,"Good quality, but could be cheaper."
4,5,2023-07-16,64,2,3,"Average experience, nothing special."


In [8]:
# approach 1
df['Score'] = df['ReviewText'].apply(lambda x: sia.polarity_scores(x)['compound'])


In [9]:
def sentiment_category(score,rating):
    if score> 0.05:
        if rating >=4:
            return 'Positive'
        elif rating == 3:
            return 'Mixed positive'
        else:
            return 'Mixed negative'
    elif score < -0.05:
        if rating <= 2:
            return 'Negative'
        elif rating == 3:
            return 'Mixed negative'
        else:
            return 'Mixed positive'
    else:
        if rating >= 4:
            return 'Positive'
        elif rating <=2:
            return 'Negative'
        else:
            return 'Neutral'
            
        

In [10]:
def sentiment_bucket (score):
    if score >= 0.5:
        return '0.5-1.0'
    elif 0.0 <= score < 0.5:
        return '0.0-0.49'
    elif -0.5< score < 0.0:
        return '-0.49-0.0'
    else:
        return '-1-0.5'

In [11]:
df['Sentiment_category']= df.apply(lambda row: sentiment_category(row['Score'],row['Rating']),axis = 1)

In [12]:
df['Sentiment_bucket'] = df['Score'].apply(lambda x:sentiment_bucket(x))

In [14]:
df

,ReviewID,ReviewDate,CustomerID,ProductID,Rating,ReviewText,Score,Sentiment_category,Sentiment_bucket
0,1,2023-12-23,77,18,3,"Average experience, nothing special.",-0.3089,Mixed negative,-0.49-0.0
1,2,2024-12-25,80,19,5,The quality is top-notch.,0.0000,Positive,0.0-0.49
2,3,2025-01-26,50,13,4,Five stars for the quick delivery.,0.0000,Positive,0.0-0.49
3,4,2025-04-21,78,15,3,"Good quality, but could be cheaper.",0.2382,Mixed positive,0.0-0.49
4,5,2023-07-16,64,2,3,"Average experience, nothing special.",-0.3089,Mixed negative,-0.49-0.0
...,...,...,...,...,...,...,...,...,...
1358,1359,2023-05-25,28,4,3,Not worth the money.,-0.1695,Mixed negative,-0.49-0.0
1359,1360,2023-11-13,58,12,2,"Average experience, nothing special.",-0.3089,Negative,-0.49-0.0
1360,1361,2023-03-07,96,15,5,Customer support was very helpful.,0.6997,Positive,0.5-1.0
1361,1362,2025-12-03,99,2,1,Product did not meet my expectations.,0.0000,Negative,0.0-0.49


In [15]:
df.to_csv('Customer_review_sentiment.csv',index = False)